In [433]:
import os
import sys
import src
import joblib
import importlib
import pandas as pd
import numpy as np
import category_encoders as ce
from src.exception import CustomException
from src.logger import logging
from typing import Union
from src.utils import fetch_data, FrequencyEncoder, Winsorizer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, TargetEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from scipy.stats.mstats import winsorize
from sklearn.ensemble import RandomForestClassifier
importlib.reload(src.utils)

<module 'src.utils' from 'f:\\data science\\ml projects\\ml project by engineering wala bhaiya\\ml_pipeline_project\\src\\utils.py'>

In [150]:
# This is an old approach to used in data insertion
'''# Setting the basepath
os.chdir('f:\\Data Science\\ML Projects\\ML Project by Engineering Wala Bhaiya\\ML_Pipeline_Project')
BASE_PATH = os.getcwd()

# Importing the dataset from the source
try:
    RAW_DATA_PATH = os.path.join(BASE_PATH, 'data', 'raw')
    income_data = pd.read_csv(os.path.join(RAW_DATA_PATH, os.listdir(RAW_DATA_PATH)[0]))
except Exception as e:
    error = CustomException(error_message=e, error_detail=sys)
    logging.info(error.error_message)
    raise e

# Strpping all columns and values from the object data
income_data.columns = income_data.columns.str.strip()
temp_df = income_data.select_dtypes(include=['object']).apply(lambda x: x.str.strip())
income_data.drop(temp_df.columns, axis=1, inplace=True)
income_data = pd.concat([income_data, temp_df], axis=1)

del(temp_df)

income_data.head()


# Visualizing the winsorization of the data
import seaborn as sns
import matplotlib.pyplot as plt
print(income_data['hours-per-week'].describe())
sns.histplot(income_data['hours-per-week'], bins=30, kde=True, color="blue")
plt.axvline(3.0, color='red', linestyle='--', label='Lower Bound (3.0)')
plt.axvline(75.0, color='green', linestyle='--', label='Upper Bound (75.0)')
plt.title('Distribution of Hours Per Week')
plt.xlabel('Hours Per Week')
plt.ylabel('Frequency')
plt.legend()
plt.show()

from scipy.stats.mstats import winsorize
import pandas as pd

# Winsorize the data
winsorized_hours = winsorize(income_data['hours-per-week'], limits=(0.001, 0.012))

# Convert the masked array back to a pandas Series for easy handling
winsorized_hours = pd.Series(winsorized_hours, index=income_data.index)

# Inspect the results
print(winsorized_hours.describe())
sns.histplot(winsorized_hours, bins=30, kde=True, color="purple")
plt.title("Distribution After Winsorization")
plt.xlabel("Hours Per Week (Winsorized)")
plt.ylabel("Frequency")
plt.show()

# Manually clip values to the 0.1% and 98.8% bounds
hours_per_week_clipped = income_data['hours-per-week'].clip(lower=3.0, upper=75.0)

# Inspect the results
print(hours_per_week_clipped.describe())
sns.histplot(hours_per_week_clipped, bins=30, kde=True, color="green")
plt.title("Manual Clipping of Hours Per Week")
plt.xlabel("Hours Per Week (Clipped)")
plt.ylabel("Frequency")
plt.show()
'''

    # This is more compact way to write a same code
'''
transform_features = [
    sublist
    for i in range(len(preprocessor.transformers_))
    for sublist in preprocessor.transformers_[i][2]
]
transform_features.extend(target_preprocessor.transformers_[0][2])
transform_features

# But this not comes into that one
transform_features = [
    item
    for sublist in [
        preprocessor.transformers_[i][2]
        for i in range(len(preprocessor.transformers_) - 1)
        ]
    for item in sublist
    ]
'''
# This is the list of random forest classifier
'''    random_forest_list = [
    RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        random_state=42,
        n_jobs=-1,
        class_weight='balanced',
        max_features='sqrt'
    )
    for _ in range(len(target_imputers))
]'''

# This code is used to calculate percentiles for winsorization (but Maunally)
'''# Calculate the 99.4th and 99.5th percentiles
lower_bound = income_data['hours-per-week'].quantile(0.001)
upper_bound = income_data['hours-per-week'].quantile(0.988)

# Extract the dataset within the specified range
print(income_data[(income_data['hours-per-week'] >= lower_bound) & (income_data['hours-per-week'] <= upper_bound)].shape,
income_data.shape[0] - income_data[(income_data['hours-per-week'] >= lower_bound) & (income_data['hours-per-week'] <= upper_bound)].shape[0]
)'''

# This code is used to extract transformed featuers
'''all_feature_names = []
for _, transformer, features in preprocessor.transformers_:
    if hasattr(transformer, 'get_feature_names_out'):
        feature_names = transformer.get_feature_names_out()
        all_feature_names.extend(feature_names)
    else:
        all_feature_names.extend(features)'''

# This code is for transforming features in train and test features seperately
'''preprocessor.fit(X_train, y_train)
# Set feature names for FrequencyEncoder
frequency_encoder = preprocessor.named_transformers_['frequency']
for name, feature in zip(['frequency', 'winsorizer'], [frequency_features, winsorize_featues]):
    preprocessor.named_transformers_[name].set_feature_names(feature)

# transform the data
X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)
print(f'Shape of X_train: {X_train_transformed.shape}', f'Shape of X_test: {X_test_transformed.shape}', sep='\n')'''

# This code is for transforming target varibale in train and test seperately
'''target_preprocessor = ColumnTransformer(transformers=[
    ('target', OrdinalEncoder(categories=[income_caregory]), ['income'])
], remainder='passthrough')
target_preprocessor.fit(y_train.to_frame(name='income'))
y_train_transformed = target_preprocessor.transform(y_train.to_frame(name='income'))
y_test_transformed = target_preprocessor.transform(y_test.to_frame(name='income'))
print(f'Shape of y_train: {y_train_transformed.shape}', f'Shape of y_test: {y_test_transformed.shape}', sep='\n')'''

# This code is use for saving the preprocess data
'''# We will continue the code from combining train and test and save them into the processed folder
BASE_PATH = os.getcwd()
DATA_PATH = os.path.join(BASE_PATH, 'data')
try:
    if os.path.exists(DATA_PATH):
        PROCESSED_PATH = os.path.join(DATA_PATH, 'processed')
        if os.path.exists(PROCESSED_PATH):
            transfromed_features = pd.concat([transforme_DataFrame(X_train_transformed), transforme_DataFrame(X_test_transformed)], axis=0)
            target_income = pd.concat([pd.Series(y_train_transformed.reshape(-1), dtype=np.int64, name='income'), pd.Series(y_test_transformed.reshape(-1), dtype=np.int64, name='income')], axis=0)
            transfrom_income_data = pd.concat([transfromed_features, target_income], axis=1)
            transfrom_income_data.to_csv(os.path.join(PROCESSED_PATH, 'Transformed_Income_Unimputed.csv'), index=False)
        else:
            print(f'Path {PROCESSED_PATH} does not exist')
    else:
        print(f'Directory {DATA_PATH} does not exist')
except OSError as e:
    print(f'OS error: {e}')'''
    
# This is code is for creating list imputive_features and target_imputers for random forest
'''# Splited features & target variables based on the missing values

# Fecthing nulled features
imputive_columns = [
    imputive_features
    for imputive_features, imputive_value in zip(
        income_data.isnull().sum().index, income_data.isnull().sum().values
        )
    if imputive_value != 0
    ]

# Creating Label Encoders for the target imputers
workclass_label_encoder, occupation_label_encoder, native_country_label_encoder = LabelEncoder(), LabelEncoder(), LabelEncoder()
encoder_list = [workclass_label_encoder, occupation_label_encoder, native_country_label_encoder]

# Creating sets of imputive features & target imputers  based on the nulled values
imputive_features = []
target_imputers = []
for imputive_feature, encoder in zip(imputive_columns, encoder_list):
    # Just have appended multiple imputive features according to the target variables
    imputive_features.append(
        transfrom_income_data.drop(columns=imputive_feature).values
    )
    
    # Applied Label Encoding on multiple targer imputers
    transformed_target = pd.concat(
        [
            pd.Series(encoder.fit_transform(X_train[imputive_feature]), name=imputive_feature),
            pd.Series(encoder.fit_transform(X_test[imputive_feature]), name=imputive_feature)
        ]
    )
    target_imputers.append(
        transformed_target.values
    )
    

print(
    f'Total Set of Imputive Features: {len(imputive_features)}',
    f'Total Set of Target Imputers: {len(target_imputers)}',
    sep='\n'
)'''

# This code is for imputining missing values using Random Forest Imputer
'''# Creating the sub-set of imputation_set where native-country has a null values
imputation_set_native_country = imputation_set[
    imputation_set['native-country'].isna() &
    ~imputation_set['workclass'].isna() &
    ~imputation_set['occupation'].isna()
].copy()


# -> Start make this as a function <-
# Transforming the sub-set where native-country has a null value
X_imputation_set_native_country = preprocessor.transform(
    imputation_set_native_country.drop(columns=['income'])
    )

# Converting them into DataFrame for droping native-country
X_imputation_set_native_country = transforme_DataFrame(
    X_imputation_set_native_country
    ).drop(columns=['native-country'])

# then transforming income
income_imputation_set_native_country = target_preprocessor.transform(
    imputation_set_native_country['income'].to_frame(name='income')
    ).ravel()

# and  in the last we are concatinating into a single set of features
X_imputation_set_native_country = pd.concat(
    [
        X_imputation_set_native_country,
        pd.Series(income_imputation_set_native_country, name='income')
        ], axis=1)

# Now put that native country subset into random forest for prediction
predictive_native_country = random_forest_list['native-country'].predict(X_imputation_set_native_country.values)
imputation_set_native_country['predictive_native_country'] = predictive_native_country

# And at the end here are the results
imputation_set_native_country.head()'''

# This code used for comparising t1 with t2
'''t1 = income_data_nona['occupation'].unique()
t2 = encoder_dict['occupation'].classes_

# Find elements in t1 that are not in t2
for i in range(len(t1)):
    is_present = False
    for j in range(len(t2)):
        if t1[i] == t2[j]:
            is_present = True
            break
    if not is_present:
        print(f"Value {t1[i]} is not present in t2")'''

# This code is used to take out the NaN values from the imputation_set based on different combunations
'''imputation_set[
    imputation_set['native-country'].isna() &
    imputation_set['workclass'].isna() &
    imputation_set['occupation'].isna()
    ].shape
    
imputation_set[
    imputation_set['workclass'].isna() &
    imputation_set['occupation'].isna() &
    ~imputation_set['native-country'].isna()
].shape

imputation_set[
    imputation_set['workclass'].isna() &
    ~imputation_set['occupation'].isna() &
    ~imputation_set['native-country'].isna()
].shape'''

"# Setting the basepath\nos.chdir('f:\\Data Science\\ML Projects\\ML Project by Engineering Wala Bhaiya\\ML_Pipeline_Project')\nBASE_PATH = os.getcwd()\n\n# Importing the dataset from the source\ntry:\n    RAW_DATA_PATH = os.path.join(BASE_PATH, 'data', 'raw')\n    income_data = pd.read_csv(os.path.join(RAW_DATA_PATH, os.listdir(RAW_DATA_PATH)[0]))\nexcept Exception as e:\n    error = CustomException(error_message=e, error_detail=sys)\n    logging.info(error.error_message)\n    raise e\n\n# Strpping all columns and values from the object data\nincome_data.columns = income_data.columns.str.strip()\ntemp_df = income_data.select_dtypes(include=['object']).apply(lambda x: x.str.strip())\nincome_data.drop(temp_df.columns, axis=1, inplace=True)\nincome_data = pd.concat([income_data, temp_df], axis=1)\n\ndel(temp_df)\n\nincome_data.head()\n\n\n# Visualizing the winsorization of the data\n"

In [2]:
income_data = fetch_data(FILE_NAME='income_evaluation.csv', DIRECTORY_NAME='raw')
income_data.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,sex,native-country,income
0,39,77516,13,2174,0,40,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,50,83311,13,0,0,13,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,38,215646,9,0,0,40,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,53,234721,7,0,0,40,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,28,338409,13,0,0,40,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K


In [3]:
income_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   fnlwgt          32561 non-null  int64 
 2   education-num   32561 non-null  int64 
 3   capital-gain    32561 non-null  int64 
 4   capital-loss    32561 non-null  int64 
 5   hours-per-week  32561 non-null  int64 
 6   workclass       32561 non-null  object
 7   education       32561 non-null  object
 8   marital-status  32561 non-null  object
 9   occupation      32561 non-null  object
 10  relationship    32561 non-null  object
 11  race            32561 non-null  object
 12  sex             32561 non-null  object
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [3]:
income_data.replace(to_replace='?', value= np.nan, inplace=True)
income_data.isnull().sum()

age                  0
fnlwgt               0
education-num        0
capital-gain         0
capital-loss         0
hours-per-week       0
workclass         1836
education            0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
native-country     583
income               0
dtype: int64

## Data Preprocessing

### 1. Encoding Caregorical Features
Here we econde the categorical features with using different encoding techniques that fits best for the each. The encoding techniques are listed following:
- One-Hot Encoding
- Ordinal Encoding
- Frequencing Encoding
- Target Encoding
<br><br>
***(Note) - Here we have tested each encoding technique, but for implementing these techniques we have to `encode` each encoder on `training set` to avoid `data leakage`.***

In [4]:
income_data.select_dtypes(include=['object']).nunique()

workclass          8
education         16
marital-status     7
occupation        14
relationship       6
race               5
sex                2
native-country    41
income             2
dtype: int64

#### 1. One Hot Encoding:
We use `One-Hot Encoding` to convert categorical features into numerical features. This technique is suitable for features that do not have a natural order.
<br><br>
**We will apply this technique on the following features**
- sex
- race

In [4]:
# Making One Hot Encoder & Selecting features
one_hot_encoder = OneHotEncoder(drop='first')
selected_features = income_data[['sex', 'race']]

# Fit and transforming the selected features
one_hot_features = one_hot_encoder.fit_transform(selected_features)
one_hot_features = one_hot_features.toarray()

# Adding the one-hot encoded features to the DataFrame
one_hot_feature_names = [
    f'{feature}_{category}'
    for feature, categories in zip(selected_features.columns, one_hot_encoder.categories_)
    for category in categories[1:]
    ]
one_hot_features = pd.DataFrame(one_hot_features, columns=one_hot_feature_names)
one_hot_features.head()

,sex_Male,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0


#### 2. Binary Encoding:
Now we implementing `Binary Encoding` to convert our target varible into numerical form.
<br><br>
**We will apply this technique on the following Target feature**
- income

<br><br><br>
***(Note) - Instead of continuing with `Binary Encoding` we use Oridinal Encoding to encode `target variable`***

In [ ]:
# Start by understanding that how this mapping works
target_binary_encoder = ce.BinaryEncoder(cols=['income'], verbose=True)
target_binary_feature = target_binary_encoder.fit_transform(income_data['income'])
target_binary_feature.head()

,income_0,income_1
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


#### 3. Ordinal Encoding:
`Ordinal encoding` is a technique used to convert categorical variables into numerical variables. It assigns a numerical value to each category based on its order or ranking.<br><br>
**We will apply this technique on the following feature**
- income (Target variable)
- education

In [22]:
# Ordering categories according to the features
education_category = [
    'Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', '10th', '11th', '12th',
    'HS-grad', 'Some-college', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Masters',
    'Prof-school', 'Doctorate'
    ]
income_caregory = ['<=50K', '>50K']

# Creating Ordinal Encoder to encode features, tramsforming & converting then into DataFrame
ordinal_encoder = OrdinalEncoder(categories=[income_caregory, education_category])
ordinal_feature = ordinal_encoder.fit_transform(income_data[['income', 'education']])
# target_ordinal_feature.reshape(-1)
ordinal_feature
ordinal_feature = pd.DataFrame(ordinal_feature, columns=['income', 'education'])
ordinal_feature.head()

,income,education
0,0.0,12.0
1,0.0,12.0
2,0.0,8.0
3,0.0,6.0
4,0.0,12.0


#### 4. Frequency Encoding:
`Frequency encoding` is a technique used in machine learning and data analysis to transform categorical data into numerical data based on the frequency of each category. This technique is useful when dealing with categorical data that has no inherent order or ranking.<br><br>
**We will apply this technique for the following fueatures**
- workclass
- occupation
- native-country

In [6]:
# Calculating the frequency of each variable
workclass_frequency = income_data['workclass'].value_counts()/len(income_data['workclass'])
occupation_frequency = income_data['occupation'].value_counts()/len(income_data['occupation'])
native_country_frequency = income_data['native-country'].value_counts()/len(income_data['native-country'])

# And then mapping the calculated frequency on each variable & converting them into DataFrame
workclass_frequency = income_data['workclass'].map(workclass_frequency)
occupation_frequency = income_data['occupation'].map(occupation_frequency)
native_country_frequency = income_data['native-country'].map(native_country_frequency)
frequency_features = pd.DataFrame({
    'worlclass': workclass_frequency,
    'occupation': occupation_frequency,
    'native-country': native_country_frequency})
frequency_features.head()

,worlclass,occupation,native-country
0,0.039864,0.115783,0.895857
1,0.078038,0.124873,0.895857
2,0.697030,0.042075,0.895857
3,0.697030,0.042075,0.895857
4,0.697030,0.127146,0.002918


In [ ]:
frequency_features.isnull().sum()

worlclass         1836
occupation        1843
native-country     583
dtype: int64

#### 5. Target Encoding:
We use `Target Encoder` to encode the target variable. This is a technique used to handle imbalanced data by encoding the target & it's also used when the categorical variable are highly coorelated with the target variable.<br><br>
**We apply this technique for the following features**
- relationship
- marital-status
<br><br><br>
<b><u><p>(Note) - `Target Encoding` contain data leakeg with in the training set if apply validation split, so we have to implement this technique while model validation</p></u></b>

In [9]:
# Initialize the TargetEncoder
target_encoder = ce.TargetEncoder(cols=['relationship', 'marital-status'])

# Fit the encoder on the training data (target column: 'income')
target_encoder.fit(income_data[['relationship', 'marital-status']], ordinal_feature['income'])

# Transform the categorical columns and add them to the dataset
target_encoder_features = target_encoder.transform(income_data[['relationship', 'marital-status']])
target_encoder_features.head()

,relationship,marital-status
0,0.103070,0.045961
1,0.448571,0.446848
2,0.103070,0.104209
3,0.448571,0.446848
4,0.475128,0.446848


### 2. Feature Scaling:

In [13]:
print(f"houre-per-week Upper Percentile: {income_data['hours-per-week'].quantile(0.988)}")
print(f"houre-per-week Lower Percentile: {income_data['hours-per-week'].quantile(0.001)}")
print(f"capital-loss Upper Percentile: {income_data['capital-loss'].quantile(0.993)}")
print(f"capital-loss Lower Percentile: {income_data['capital-loss'].quantile(0.0)}")
print(f"capital-gain Upper Percentile: {income_data['capital-gain'].quantile(0.993)}")
print(f"capital-gain Lower Percentile: {income_data['capital-gain'].quantile(0.05)}")

houre-per-week Upper Percentile: 75.0
houre-per-week Lower Percentile: 3.0
capital-loss Upper Percentile: 2174.0
capital-loss Lower Percentile: 0.0
capital-gain Upper Percentile: 20051.0
capital-gain Lower Percentile: 0.0


In [26]:
print(
    winsorize(income_data['hours-per-week'].values, limits=(0.001, 0.012)).data,
    winsorize(income_data['capital-gain'].values, limits=(0.05, 0.007)).data,
    winsorize(income_data['capital-loss'].values, limits=(0.00, 0.007)).data,
    sep='\n'
    )

[40 13 40 ... 40 20 40]
[ 2174     0     0 ...     0     0 15024]
[0 0 0 ... 0 0 0]


### 3. Feature Transformation
There we will be using SK-Learn ColumnTransformer to transform the data.

In [4]:
# Sates of the missing and non-missing set
print(f'Size of the orignal income set: {income_data.shape}')
print(f'Size of missing set: {income_data[income_data.isna().any(axis=1)].shape}')
print(f'Size of the non-missing set: {income_data.dropna().shape}')
print(f'Total size of the missing and non-missing set: ({income_data[income_data.isna().any(axis=1)].shape[0] + income_data.dropna().shape[0]}, {income_data.dropna().shape[1]})')

Size of the orignal income set: (32561, 15)
Size of missing set: (2399, 15)
Size of the non-missing set: (30162, 15)
Total size of the missing and non-missing set: (32561, 15)


In [131]:
# Spliting income set based on the missing values for imputation
imputation_set = income_data[income_data.isna().any(axis=1)].copy()
income_data_nona = income_data.dropna().copy()

# Split the income data into training and test set
X_train, X_test, y_train, y_test = train_test_split(income_data_nona.drop(columns=['income']), income_data_nona['income'], test_size=0.15, random_state=42)
print(f'Shape of training features:  {X_train.shape}, Shape of training labels: {y_train.shape}')
print(f'Shape of testing features:  {X_test.shape}, Shape of testing labels: {y_test.shape}')
print(f'Total training and test smaple: ({X_train.shape[0] + X_test.shape[0]}, {X_train.shape[1]})')

Shape of training features:  (25637, 14), Shape of training labels: (25637,)
Shape of testing features:  (4525, 14), Shape of testing labels: (4525,)
Total training and test smaple: (30162, 14)


In [47]:
# Preprocessing for each encoding technique
onehot_features = ['sex', 'race']
ordinal_features = ['education']
frequency_features = ['workclass', 'occupation', 'native-country']
target_features = ['relationship', 'marital-status']
winsorize_featues = ['hours-per-week', 'capital-gain', 'capital-loss']
limits = [(0.001, 0.988), (0.05, 0.993), (0.00, 0.993)]
feature_limits = {feature: limit for feature, limit in zip(winsorize_featues, limits)}
income_caregory = ['<=50K', '>50K']
remaining_features = income_data.columns[:3].tolist()

In [258]:
# Define preprocessing pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop='first'), onehot_features),
        ('ordinal', OrdinalEncoder(), ordinal_features),
        ('frequency', FrequencyEncoder(), frequency_features),
        ('target', TargetEncoder(), target_features),
        ('winsorizer', Winsorizer(feature_limits=feature_limits), winsorize_featues),
        ('remainders', 'passthrough', remaining_features),
    ],
    remainder='passthrough' # Keep other columns as-is (numetical features)
)

In [260]:
X_train = income_data_nona.drop(columns=['income'])
y_train = income_data_nona['income']
preprocessor.fit(X_train, y_train)
# Set feature names for FrequencyEncoder
# frequency_encoder = preprocessor.named_transformers_['frequency']
for name, feature in zip(['frequency', 'winsorizer'], [frequency_features, winsorize_featues]):
    preprocessor.named_transformers_[name].set_feature_names(feature)

# transform the data
X_train_transformed = preprocessor.transform(X_train)
# X_test_transformed = preprocessor.transform(X_test)
print(f'Shape of the Features in Income Dataset: {X_train_transformed.shape}')

Shape of the Features in Income Dataset: (30162, 17)


In [10]:
def transforme_DataFrame(X_transformed):
    all_feature_names = []
    for _, transformer, features in preprocessor.transformers_:
        if hasattr(transformer, 'get_feature_names_out'):
            feature_names = transformer.get_feature_names_out()
            all_feature_names.extend(feature_names)
        else:
            all_feature_names.extend(features)
    exclude_features = len(features)

    # X_train_transformed[:, :-temp].shape, len(all_feature_names)
    # return pd.DataFrame(X_transformed[:, :-exclude_features], columns=all_feature_names[:-exclude_features])
    return pd.DataFrame(X_transformed, columns=all_feature_names)

In [261]:
# This Cell is For Experimentation
y_train = income_data_nona['income']
target_preprocessor = ColumnTransformer(transformers=[
    ('target', OrdinalEncoder(categories=[income_caregory]), ['income'])
], remainder='passthrough')
target_preprocessor.fit(y_train.to_frame(name='income'))
y_train_transformed = target_preprocessor.transform(y_train.to_frame(name='income'))
# y_test_transformed = target_preprocessor.transform(y_test.to_frame(name='income'))
print(f'Shape of Target in Income Dataset: {y_train_transformed.shape}')

Shape of Target in Income Dataset: (30162, 1)


In [202]:
pd.Series(y_train_transformed.reshape(-1), dtype=np.int64, name='income').to_frame('income').head()

,income
0,0
1,0
2,0
3,0
4,0


## Saving Processed Data
There we have `saved unimputed data`, that gonna use later in the Random Forest to learning imputation
<br><br><br>
<b><u><i>Note - Please do not run the following cell again. If you don't the file then you could</i></u></b>

In [206]:
# This Cell is For Experimentation

# We will continue the code from combining train and test and save them into the processed folder
BASE_PATH = os.getcwd()
DATA_PATH = os.path.join(BASE_PATH, 'data')
try:
    if os.path.exists(DATA_PATH):
        PROCESSED_PATH = os.path.join(DATA_PATH, 'processed')
        if os.path.exists(PROCESSED_PATH):
            transfrom_income_data = pd.concat(
                [
                    transforme_DataFrame(X_train_transformed),
                    pd.Series(y_train_transformed.ravel(), name='income')],
                axis=1)
            transfrom_income_data.to_csv(
                os.path.join(PROCESSED_PATH,'Transformed_Income_Unimputed_Without_NaN.csv'),
                index=False)
        else:
            print(f'Path {PROCESSED_PATH} does not exist')
    else:
        print(f'Directory {DATA_PATH} does not exist')
except OSError as e:
    print(f'OS error: {e}')

In [15]:
# print(f"Transformed Data Shape: {transfrom_income_data.shape}")

Transformed Data Shape: (30162, 18)


## Hendling Missing Values

### 1. Imputetaion with Random Forest
While will use `Random Forest` to impute missing values, and which is also a populer approach handle missing values, The idea is to `Train Random Forest` on non missing data, and to use `model to impute` the data that contain missing values.

#### 1. Fetch Processed Data

In [208]:
transfrom_income_data = fetch_data(FILE_NAME='Transformed_Income_Unimputed_Without_NaN.csv', DIRECTORY_NAME='processed')
transfrom_income_data.head(5)

,sex_Male,race_Asian-Pac-Islander,race_Black,race_Other,race_White,education,workclass,occupation,native-country,relationship,marital-status,hours-per-week,capital-gain,capital-loss,age,fnlwgt,education-num,income
0,1.0,0.0,0.0,0.0,1.0,9.0,0.042404,0.123367,0.911876,0.106533,0.048329,40.0,2174.0,0.0,39.0,77516.0,13.0,0.0
1,1.0,0.0,0.0,0.0,1.0,9.0,0.082853,0.132352,0.911876,0.455647,0.454940,13.0,0.0,0.0,50.0,83311.0,13.0,0.0
2,1.0,0.0,0.0,0.0,1.0,11.0,0.738877,0.044758,0.911876,0.106533,0.107279,40.0,0.0,0.0,38.0,215646.0,9.0,0.0
3,1.0,0.0,1.0,0.0,0.0,1.0,0.738877,0.044758,0.911876,0.455647,0.454940,40.0,0.0,0.0,53.0,234721.0,7.0,0.0
4,0.0,0.0,1.0,0.0,0.0,9.0,0.738877,0.133877,0.003050,0.493366,0.454940,40.0,0.0,0.0,28.0,338409.0,13.0,0.0


#### 2. Created a Set of Features and Target Varibale
There we are creating set of features and target varibale in a form of list, where in the list each feature set corrispond to each target varibale.

In [209]:
# Splited features & target variables based on the missing values

# Fecthing nulled features
imputive_columns = [
    imputive_features
    for imputive_features, imputive_value in zip(
        income_data.isnull().sum().index,
        income_data.isnull().sum().values
        )
    if imputive_value != 0
    ]

# Creating Label Encoders for the target imputers
workclass_label_encoder, occupation_label_encoder, native_country_label_encoder = LabelEncoder(), LabelEncoder(), LabelEncoder()
encoder_dict = {'workclass': workclass_label_encoder, 'occupation': occupation_label_encoder, 'native_country': native_country_label_encoder}

# Creating sets of imputive features & target imputers  based on the nulled values
imputive_features = []
target_imputers = []
for imputive_feature, encoder_key in zip(imputive_columns, encoder_dict.keys()):
    
    # Just have appended multiple imputive features according to the target variables
    imputive_features.append(
        transfrom_income_data.drop(columns=imputive_feature).values
    )
    
    # Applied Label Encoding on multiple targer imputers
    target_imputers.append(
        encoder_dict[encoder_key].fit_transform(income_data_nona[imputive_feature])
    )
    

print(
    f'Total Set of Imputive Features: {len(imputive_features)}',
    f'Total Set of Target Imputers: {len(target_imputers)}',
    sep='\n'
)

Total Set of Imputive Features: 3
Total Set of Target Imputers: 3


In [165]:
imputive_features[0].shape, target_imputers[0].shape

((30162, 17), (30162,))

#### 3. Make a Function that Impute Each Targer Variable in a Set
This `imputation_random_forest` takes different set of features and target variables and then impute each target variable using random forest imputation. It returns a dictionary with the imputed target.

In [167]:
def impute_random_forest(
    features_list: Union[list, np.ndarray],
    targets_list: Union[list, np.ndarray],
    imputive_columns: Union[str, list]) -> dict:
    
    """
    Fits a list of Random Forest models to impute target values.
    
    Parameters:
    - features_list: list of numpy arrays or pandas DataFrames (features for each target variable)
    - targets_list: list of numpy arrays or pandas Series (target variables to impute)

    Returns:
    - random_forest_dict: dictionary of trained Random Forest models
    """
    
    # Creating empty list to store the random forest models
    random_forest_dict = {}
    
    # Check the validation of all the input parameters - In term of lenght
    if not (len(features_list) == len(targets_list) == len(imputive_columns)):
        raise ValueError("Input lists must have the same length.")
    
    # Train multiple Random Forest models
    for index, (X_train, y_train, imputive_variable) in enumerate(
        zip(features_list, targets_list, imputive_columns)
        ):
        # Creating the random forest model
        random_forest = RandomForestClassifier(
            n_estimators=200,
            max_depth=15,
            random_state=42,
            n_jobs=-1,
            class_weight='balanced',
            max_features='sqrt',
            verbose=1
        )
        
        # Fitting the model to the training data
        random_forest.fit(X_train, y_train)
        
        # Appending the model to the list
        random_forest_dict[imputive_variable] = random_forest
    return random_forest_dict

#### 4. Train the model
Now we have all the components, we can train our `Random Forst` based on the list of features that corrispond to the list of target variable

In [ ]:
random_forest_dict = impute_random_forest(imputive_features, target_imputers, imputive_columns)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    4.2s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    6.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    5.7s finished


#### 5. Impute Missing Values
Then we impute missing values by using Random Forest that we have train before. and we will create combunation of feature sets that don't have target feature.
+ We seperate features that have to `impute single value` in a row
+ We seperate features that have to `impute multiple values` in a row
+ We use Random Forest to `impute missing values` in each feature set
+ Then we combine all imputed DataFrames in one DataFrame name `imputation_set`.
+ Finally we combine `imputation_set` (contain imputed values in the features insted of missing one) with the `income_data_nona` (contain no missing values) and we get `imputed_income_data` DataFrame.

In [308]:
def impute_target_variable(X_imputive_set: pd.DataFrame, target_feature: str) -> pd.DataFrame:
    # Transforming all the features including null and not null values
    X_imputation_set = preprocessor.transform(
        X_imputive_set.drop(columns=['income'])
        )

    # Converting them into DataFrame to drop target variable
    X_imputation_set = transforme_DataFrame(
        X_imputation_set
        ).drop(columns=[target_feature])

    # then transforming income
    income_imputation_set_native_country = target_preprocessor.transform(
        X_imputive_set['income'].to_frame(name='income')
        ).ravel()

    # and  in the last we are concatinating into a single set of features
    X_imputation_set = pd.concat(
        [
            X_imputation_set,
            pd.Series(income_imputation_set_native_country, name='income')
            ], axis=1
        )

    # Now put that target variable into random forest for prediction
    predictive_feature = random_forest_list[target_feature].predict(X_imputation_set.values)
    X_imputive_set[str('predictive_' + target_feature)] = predictive_feature
    X_imputive_set = X_imputive_set[str('predictive_' + target_feature)].to_frame(name=str('predictive_' + target_feature))
    return X_imputive_set

In [ ]:
def impute_multi_target(X_imputive_set: pd.DataFrame, target_variables: Union[list, np.array]) -> pd.DataFrame:
    X_imputed_list = []
    for target in target_variables:
        X_imputed = impute_target_variable(X_imputive_set=X_imputive_set, target_feature=target)
        X_imputed_list.append(X_imputed)
    # Concatenate the list of DataFrames
    X_imputed_concat = pd.concat(X_imputed_list, axis=1)
    # Drop duplicate columns if any
    X_imputive_set = X_imputed_concat.loc[:,~X_imputed_concat.columns.duplicated()]
    return X_imputive_set

In [339]:
imputation_set_occupation = imputation_set[
    imputation_set['occupation'].isna() &
    ~imputation_set['workclass'].isna() &
    ~imputation_set['native-country'].isna()
].copy()
imputation_set_occupation = impute_target_variable(imputation_set_occupation, 'occupation')
imputation_set_occupation['predictive_occupation'] = encoder_dict['occupation'].inverse_transform(imputation_set_occupation['predictive_occupation'])
imputation_set_occupation

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.0s finished


,predictive_occupation
5361,Farming-fishing
10845,Adm-clerical
14772,Other-service
20337,Adm-clerical
23232,Protective-serv
32304,Adm-clerical
32314,Farming-fishing


<u><i>Features that have to impute single value in a row</i></u>

In [347]:
# We will start the code by testing function -> impute_target_variable
imputation_set_native_country = imputation_set[
    imputation_set['native-country'].isna() &
    ~imputation_set['workclass'].isna() &
    ~imputation_set['occupation'].isna()
].copy()
imputation_set_native_country = impute_target_variable(imputation_set_native_country, 'native-country')
# imputation_set_native_country
imputation_set_native_country['predictive_native-country'] = encoder_dict['native_country'].inverse_transform(imputation_set_native_country['predictive_native-country'])
imputation_set_native_country.head()

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.1s finished


,predictive_native-country
14,Philippines
38,United-States
51,United-States
93,Vietnam
245,United-States


<u><i>Features that have to impute multiple value in a row</i></u>

In [349]:
imputation_set_workclass_occupation = imputation_set[
    imputation_set['workclass'].isna() &
    imputation_set['occupation'].isna() &
    ~imputation_set['native-country'].isna()
].copy()
imputation_set_workclass_occupation = impute_multi_target(imputation_set_workclass_occupation, ['workclass', 'occupation'])
# imputation_set_workclass_occupation
for f, col in zip(imputation_set_workclass_occupation.columns, ['workclass', 'occupation']):
    imputation_set_workclass_occupation[f] = encoder_dict[col].inverse_transform(imputation_set_workclass_occupation[f])
imputation_set_workclass_occupation.head()

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.4s finished


,predictive_workclass,predictive_occupation
27,Private,Adm-clerical
69,Federal-gov,Adm-clerical
77,Private,Craft-repair
106,Private,Other-service
128,State-gov,Adm-clerical


In [353]:
imputation_set_workclass_occupation_country = imputation_set[
    imputation_set['native-country'].isna() &
    imputation_set['workclass'].isna() &
    imputation_set['occupation'].isna()
    ].copy()
imputation_set_workclass_occupation_country = impute_multi_target(
    imputation_set_workclass_occupation_country,
    ['workclass', 'occupation', 'native-country']
    )
for f, col in zip(imputation_set_workclass_occupation_country.columns, ['workclass', 'occupation', 'native_country']):
    imputation_set_workclass_occupation_country[f] = encoder_dict[col].inverse_transform(imputation_set_workclass_occupation_country[f])
imputation_set_workclass_occupation_country.head()

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.0s finished


,predictive_workclass,predictive_occupation,predictive_native-country
61,Private,Craft-repair,Mexico
297,Private,Prof-specialty,United-States
1152,Private,Handlers-cleaners,United-States
1676,Private,Tech-support,United-States
2513,Private,Adm-clerical,United-States


In [ ]:
def set_predictors_with_imputation_set(imputation_set, predictor_set, features: list):
    existing_values = imputation_set.loc[predictor_set.index, features].notna().any().any()
    if existing_values:
        raise ValueError(
            "Assignment halted: One or more target features in imputation_set already have non-NaN values."
        )
    predictive_features = ['predictive_' + feature for feature in features]
    imputation_set.loc[predictor_set.index, features] = predictor_set[predictive_features].values
    return imputation_set

***Note - Please ensure that you run the following just a once only***

In [408]:
try:
    predictive_imputive_set_list = [
        imputation_set_occupation,
        imputation_set_native_country,
        imputation_set_workclass_occupation,
        imputation_set_workclass_occupation_country
        ]
    predictive_imputive_features_list = [
        ['occupation'],
        ['native-country'],
        ['workclass', 'occupation'],
        ['workclass', 'occupation', 'native-country']
    ]
    for predictor_set, col in zip(predictive_imputive_set_list,predictive_imputive_features_list):
        imputation_set = set_predictors_with_imputation_set(imputation_set, predictor_set, col)
except ValueError as e:
    error = CustomException(e,sys)
    logging.info(error.error_message)
imputation_set.isna().sum()

age               0
fnlwgt            0
education-num     0
capital-gain      0
capital-loss      0
hours-per-week    0
workclass         0
education         0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
native-country    0
income            0
dtype: int64

In [430]:
imputed_income_data = pd.concat([income_data_nona, imputation_set], axis=0).sort_index()

#### 6. Save Imputed Dataset & Imputer Model
At the last we save imputed income data as a CSV formate into data/processed directory & Dictionay of Models

##### 1. Save Dataset
We have save a dataset in CSV format.

In [432]:
imputed_income_data.to_csv(
    'F:\Data Science\ML Projects\ML Project by Engineering Wala Bhaiya\ML_Pipeline_Project\data\processed\Imputed_Income_Dataset_RF.csv',
    index=False)

##### 2. Save Models
We have saved the model in the form of dictionary using joblib.

In [437]:
with open('models/Random_Forest_Imputuer_Dict.pkl', 'wb') as file:
    joblib.dump(random_forest_dict, file)